## Parent Paper Code Run
### Compose and Embellish: Well-Structured Piano Performance Generation via A Two-Stage Approach
#### 17 Sep 2022  ·  Shih-Lun Wu, Yi-Hsuan Yang

### Team 9: Brian Ellis,

In [18]:
# Clone repository
!git clone https://github.com/brianellis1997/Music_Generation.git
%cd Music_Generation

# Install libraries
!pip install -r requirements.txt

# Install pre-trained transformers (15 min runtime)
!pip install git+https://github.com/cifkao/fast-transformers.git@39e726864d1a279c9719d33a95868a4ea2fb5ac5
!git clone https://huggingface.co/slseanwu/compose-and-embellish-pop1k7
!pip install miditoolkit

Cloning into 'Music_Generation'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 117 (delta 50), reused 36 (delta 7), pack-reused 0
Receiving objects: 100% (117/117), 897.48 KiB | 7.74 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Music_Generation/Music_Generation/Music_Generation


In [19]:
# Compose

# Generating a leadsheet
!python3 stage01_compose/inference.py \
  stage01_compose/config/pop1k7_finetune.yaml \
  generation/stage01 \
  1   # Generate one leadsheet

Please input max_bars: 64
Please input temperature (randomness): 1.4
Please input top_p: 1.3
Invalid input. Please enter a valid numeric value.
Please input top_p: 2
[nucleus parameters] t = 1.4, p = 2
[info] # params: 41331059
Traceback (most recent call last):
  File "/content/Music_Generation/Music_Generation/Music_Generation/stage01_compose/inference.py", line 131, in <module>
    pretrained_dict = torch.load(config['inference_param_path'], map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 986, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 435, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 416, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'compose-and-embellish-pop1k7/compose_model_lmd_pretra

In [12]:
# Generate multiple lead sheets with difference of temperature
for i in range(1, 6):
  !python3 stage01_compose/inference.py \
    stage01_compose/config/pop1k7_finetune.yaml \
    generation/stage01 \
    1   # Generate one leadsheet

Please input max bars: 32
Please input temperature (randomness): 1
[nucleus parameters] t = 1.0, p = 0.97
[info] # params: 41331059
[global tempo] 119
 -- generating leadsheet #1 of 1
[info] generated 1 bars, #events = 15
[info] generated 2 bars, #events = 26
[info] generated 3 bars, #events = 39
[info] generated 4 bars, #events = 52
[info] generated 5 bars, #events = 65
[info] generated 6 bars, #events = 78
[info] generated 7 bars, #events = 93
[info] generated 8 bars, #events = 104
[info] generated 9 bars, #events = 116
[info] generated 10 bars, #events = 127
[info] generated 11 bars, #events = 140
[info] generated 12 bars, #events = 153
[info] generated 13 bars, #events = 169
[info] generated 14 bars, #events = 182
[info] generated 15 bars, #events = 197
[info] generated 16 bars, #events = 210
[info] generated 17 bars, #events = 227
[info] generated 18 bars, #events = 238
[info] generated 19 bars, #events = 249
[info] generated 20 bars, #events = 260
[info] generated 21 bars, #event

In [ ]:
# Embellish

# We will embellish our generated leadsheet
!python3 stage02_embellish/inference.py \
  stage02_embellish/config/pop1k7_default.yaml \
  generation/stage01 \
  generation/stage02

[preparing data] now at #0
[info] model init completed
[info] model loaded
[# pieces] 1
/usr/local/lib/python3.10/dist-packages/fast_transformers/feature_maps/fourier_features.py:37: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2426.)
  Q, _ = torch.qr(block)
[info] generated 1 bars, #events = 86
[info] generated 2 bars, #events = 220
[info] generated 3 bars, #events = 366
[info] generated 4 bars, #events = 440
[info] generated 5 bars, #events = 522
[info] generated 6 bars, #events = 665
[info] generated 7 bars, #events = 788
[info] generated 8 bars, #events = 910
[info] generated 9 bars, #events = 1016
[info] generated 10 bars, #events = 1138
[info] generated 11

In [4]:
# installs and imports to convert MIDI into audio
!pip install pretty_midi
!wget https://www.dropbox.com/s/4x27l49kxcwamp5/GeneralUser_GS_1.471.zip
!unzip GeneralUser_GS_1.471.zip
!apt install -y fluidsynth
from pretty_midi import PrettyMIDI
from IPython.display import Audio
from scipy.io.wavfile import write
import librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592284 sha256=268a49e04a0082a217ff2f3b5c735e0a7e4537f5918e08830be108296e36f0d9
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
--2024-03-07 15:14:39--  https://www.dropbox.com/s/4x27l49kxcwamp5/GeneralUser_GS_1.471.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4x27l49kxcwamp5/GeneralUser_GS_1.471.zip [following]
--2024-03-07 15:14:40--  https://www.dropbox.com/s/raw/4x27l49kxcwamp5/GeneralUser_GS_1.471.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
L

In [9]:
##########
# LISTEN #
##########

# render the first stage
!fluidsynth -ni GeneralUser\ GS\ 1.471/GeneralUser\ GS\ v1.471.sf2 generation/stage01/samp_01.mid -F first_stage.wav -r 44100

# render the second stage
# !fluidsynth -ni GeneralUser\ GS\ 1.471/GeneralUser\ GS\ v1.471.sf2 generation/stage02/samp_01_2stage_samp01.mid -F second_stage.wav -r 44100

# # uncomment if you want to hear the melody
# # generated in the first stage
# # hear the first stage
x,sr=librosa.load('first_stage.wav')
Audio(x,rate=sr)

# hear the second stage
# x,sr=librosa.load('second_stage.wav')
# Audio(x,rate=sr)

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'first_stage.wav'..
